In [169]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

This code is written as part of COMP8755- Individual Computing Project

This code is for preprocessing and training an LSTM network

The songs are first parsed and the notes of different instruments are procesed to train and mdh5 files for music generation.


@Author:
Mithun
u6849970

@ Supervisor:
Professor Nick Birbillis

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


'\n\nThis code is written as part of COMP8755- Individual Computing Project\n\nThis code is for preprocessing and training an LSTM network\n\nThe songs are first parsed and the notes of different instruments are procesed to train and mdh5 files for music generation.\n\n\n@Author:\nMithun\nu6849970\n\n@ Supervisor:\nProfessor Nick Birbillis\n\n'

In [170]:
from music21 import *
import os
from keras.utils import np_utils
import numpy as np
from keras.layers import *
from keras.models import Sequential
from collections import Counter

In [171]:
"""
This function is for reading the midi files which we feed as a data set and 
parse it into 3 categories, chords, rests and notes
"""
def midi_read(file):
    print("Loading File:",file)
    
    note=[]
    notes_to_parse = None
    rest = True
    #parsing a midi file
    midi = converter.parse(file)
    song = instrument.partitionByInstrument(midi)
    print("Number of instrument parts: " + str(len(s2.parts)))
    for part in song.parts:
          
        notes_to_parse = part.recurse() 
      
        #finding whether a particular element is note or a chord    
        for element in notes_to_parse:                
                #note
            if isinstance(element, note.Note):
                note.append(str(element.pitch))
                #rest
            elif isinstance(element, note.Rest) and rest:
                note.append("rest")                
                #chord
            elif isinstance(element, chord.Chord):
                note.append('.'.join(str(n) for n in element.normalOrder))

    return np.array(note)

    return note

In [178]:
path='midi/'

#read all the filenames
files=[i for i in os.listdir(path) if i.endswith(".mid")]


#reading each midi file
notes_array = np.array([read_midi(path+i) for i in files])

Loading Music File: midi/Recognizer.mid
Number of instrument parts: 7
Loading Music File: midi/Daft_Punk_Medley_Pentatonix_Full_Arrangement_SATB.mid
Number of instrument parts: 2
Loading Music File: midi/GiorgiobyMoroder.mid
Number of instrument parts: 1
Loading Music File: midi/AroundTheWorld.mid
Number of instrument parts: 5
Loading Music File: midi/TheGrid.mid
Number of instrument parts: 4
Loading Music File: midi/FlynnLives.mid
Number of instrument parts: 11
Loading Music File: midi/Harder_Better_Faster_Stronger_-_Daft_Punk.mscz.mid
Number of instrument parts: 1
Loading Music File: midi/Veridisquo.mid
Number of instrument parts: 5
Loading Music File: midi/Contact.mid
Number of instrument parts: 7
Loading Music File: midi/SomethingAboutUs.mid
Number of instrument parts: 4
Loading Music File: midi/ShortCircuit.mid
Number of instrument parts: 6
Loading Music File: midi/RobotRock.mid
Number of instrument parts: 4
Loading Music File: midi/Aerodynamic(2).mid
Number of instrument parts: 5

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [181]:
#converting 2D array into a vector
notes = [element for note_ in notes_array for element in note_]
for note_ in notes_array:
    for element in note_:
        notes.append(element)
   
#No. of unique notes
uniqueNotes = list(set(notes))
print(len(uniqueNotes))

302


In [182]:
#we use the frequent notes to generate music
freq = dict(Counter(notes_))

for note_,count in freq.items():
    if count>=50:
        frequent_notes.append(note_)
#frequent_notes = [note_ for note_, count in freq.items() if count>=60]
print(len(frequent_notes))

252


In [183]:
newMusic=[]

for notes in notes_array:
    temp = [note for note in notes if note in frequent_notes]
    """for note in notes:
        if note in frequent_notes:
            temp.append(note)  """          
    newMusic.append(temp)
    
newMusic = np.array(newMusic)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.


In [184]:
#timesteps is a function to tell the number of input notes given for generating the output
no_of_timesteps = 25
x = []
y = []

for note_ in new_music:
    for i in range(0, len(note_) - no_of_timesteps, 1):
        
        #preparing input and output sequences
        input_ = note_[i:i + no_of_timesteps]
        output = note_[i + no_of_timesteps]
        
        x.append(input_)
        y.append(output)
        
x=np.array(x)
y=np.array(y)

In [185]:
#reshaping array to 1D
unique_x = list(set(x.ravel()))
x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

In [186]:
#preparing input sequences
x_seq=[]
for i in x:
    temp=[x_note_to_int[j] for j in i]
    x_seq.append(temp)
    
x_seq = np.array(x_seq)

In [140]:
unique_y = list(set(y))
y_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_y)) 
y_seq=np.array([y_note_to_int[i] for i in y])

In [141]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_seq,y_seq,test_size=0.2,random_state=0)

In [142]:
"""
we need to create the network back so that we can predict the next set of notes for generating music
"""

'\nwe need to create the network back so that we can predict the next set of notes for generating music\n'

In [143]:
#Single layer LSTM model with activation as softmax and optimizer as adam
def lstm():
   model = Sequential()
   model.add(Dropout(0.3))
   model.add(LSTM(512, return_sequences=True))
   model.add(Dropout(0.3))
   model.add(Dense(n_vocab))
   model.add(Activation('softmax'))
   model.compile(loss='categorical_crossentropy', optimizer='adam')
   return model

In [144]:
"""
This is the checkpint for training the model, after every epoch,
the model is updated. We can stop the model once the loss value has saturated
"""
from keras.callbacks import ModelCheckpoint
mc= ModelCheckpoint('best_model.h5',monitor='val_loss',mode='min',save_best_only=True,verbose=1)


In [106]:

history = model.fit(np.array(x_tr),np.array(y_tr),batch_size=128,epochs=100, validation_data=(np.array(x_val),np.array(y_val)),verbose=1, callbacks=[mc])

Epoch 1/100
 77/128 [=================>............] - ETA: 3s - loss: 0.8233

KeyboardInterrupt: 

In [145]:
"""
This function generates the notes in a sequential pattern which is then converted to MIDI file
we pick a random sequence from the input and use it as a starting point for generation
"""

'\nThis function generates the notes in a sequential pattern which is then converted to MIDI file\nwe pick a random sequence from the input and use it as a starting point for generation\n'

In [146]:
import random
ind = np.random.randint(0,len(x_val)-1)
"""Random number for prediction
"""
random_music = x_val[ind]

predictions=[]
"""
a total of 150 notes are being generated for our dataset,
this value can be changed depending on the dataset.

"""
sequence_length = 500
for i in range (sequence_length):

    random_music = random_music.reshape(1,no_of_timesteps)

    #from the prediction input, we use the keras libary for the same

    prob  = model.predict(random_music)[0]
    y_pred= np.argmax(prob,axis=0)
    #The pattern will contain the musical notes which is stored as an array
    predictions.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]
    
print(predictions)

[65, 43, 24, 43, 43, 26, 21, 22, 43, 7, 43, 43, 21, 7, 68, 15, 22, 22, 22, 70, 54, 54, 54, 3, 57, 7, 54, 60, 54, 50, 50, 56, 48, 12, 48, 62, 62, 62, 19, 54, 54, 64, 64, 54, 54, 54, 30, 72, 72, 64, 64, 64, 52, 52, 63, 66, 66, 43, 57, 57, 57, 43, 47, 63, 43, 43, 30, 54, 54, 70, 41, 7, 54, 54, 48, 21, 14, 54, 7, 46, 30, 46, 46, 7, 7, 21, 7, 45, 7, 39, 7, 54, 54, 26, 54, 54, 3, 3, 22, 70, 65, 50, 7, 50, 7, 18, 50, 70, 46, 54, 50, 30, 14, 14, 14, 54, 7, 52, 14, 7, 50, 7, 3, 48, 3, 70, 7, 7, 3, 3, 50, 50, 50, 50, 50, 50, 50, 3, 50, 50, 50, 50, 50, 50, 3, 3, 48, 7, 46, 11, 7, 52, 7, 43, 7, 7, 7, 65, 7, 7, 7, 50, 66, 50, 50, 50, 70, 50, 50, 3, 50, 3, 3, 3, 3, 50, 54, 30, 54, 54, 54, 54, 7, 45, 22, 7, 43, 43, 43, 43, 30, 30, 30, 30, 43, 64, 54, 30, 7, 54, 54, 7, 7, 54, 54, 46, 46, 11, 7, 46, 7, 7, 7, 7, 7, 7, 24, 3, 54, 57, 54, 54, 57, 54, 50, 3, 3, 30, 50, 50, 66, 50, 50, 50, 54, 48, 50, 24, 54, 54, 66, 43, 54, 54, 54, 54, 30, 5, 22, 63, 63, 22, 22, 64, 52, 64, 64, 64, 52, 52, 64, 30, 43, 57, 

In [147]:
x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x)) 
predicted_notes = [x_int_to_note[i] for i in predictions]

In [148]:
"""
Generating the Midi file from prediction
"""

'\nGenerating the Midi file from prediction\n'

In [128]:
"""

“We used the Generate Piano Instrumental Music by Using Deep Learning(2019) by Haryo Akbarianto Wibowo to complete our work."

""""

def create_midi(prediction_output):
    offset = 0
    output_notes = []

    

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        elif pattern == 'rest':
            new_note = note.Rest()
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.25

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='music.mid')



In [129]:
create_midi(predicted_notes)